# Imports

Author: Zhongyang Zhang

Email : mirakuruyoo@gmail.com

In [1]:
import glob
import os
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from skimage.feature import hog
from skimage.feature import haar_like_feature
from skimage.feature import local_binary_pattern as lbp
from skimage.io import imread
from skimage.transform import pyramid_gaussian, integral_image
from sklearn.externals import joblib

%matplotlib inline

# Configs 
Please change the PROJECT_ID term every time when excuting a new experiment

In [17]:
class Config(object):
    def __init__(self, project_id=None):
        
        # General Config
        self.DES_TYPE = "HOG"
        self.CLF_TYPE = "LIN_SVM"
        if project_id:
            self.PROJECT_ID = project_id
        else:
            self.PROJECT_ID = "New_Vedio_New_Neg" + self.DES_TYPE + '_' + self.CLF_TYPE
        self.THRESHOLD = 0.3
        self.DOWNSCALE = 1.25
        
        # Pathes
        self.update_names()
        
        # HOG Features
        self.MIN_WDW_SIZE = [64, 64]
        self.STEP_SIZE = [12, 12]
        self.ORIENTATIONS = 9
        self.PIXELS_PER_CELL = [3, 3]
        self.CELLS_PER_BLOCK = [3, 3]
        self.VISUALIZE = False
        self.NORMALIZE = True
        self.IF_PRINT  = False
        self.KEEP_FEAT = False
        
        # LBP Features
        self.LBP_RADIUS = 3
        self.LBP_POINTS = 8 * self.LBP_RADIUS
        
        self.mk_new_dirs()
        
    def mk_new_dirs(self):
        for ph in self.DIR_PATHS.values():
            if not os.path.exists(ph):
                os.makedirs(ph)
                print("==> Directory Tree",ph,"created")
                
    def update_names(self):
        # Pathes
        self.DIR_PATHS = {
            "POS_FEAT_PH"    : os.path.join("./source/features", self.PROJECT_ID,"pos"),
            "NEG_FEAT_PH"    : os.path.join("./source/features", self.PROJECT_ID,"neg"),
            "MODEL_DIR_PH"   : os.path.join("./source/models", self.PROJECT_ID),
            "PRED_SAVE_PH"   : os.path.join("./source/predictions", self.PROJECT_ID),
            "POS_IMG_PH"     : "./source/images/pos",
            "NEG_IMG_PH"     : "./source/images/neg",
            "TEST_IMG_DIR_PH": "./source/test_images"}
        self.MODEL_PH = os.path.join(self.DIR_PATHS["MODEL_DIR_PH"], "svm.model")
        self.TEST_IMG_PH = os.path.join(self.DIR_PATHS["TEST_IMG_DIR_PH"], "test.jpg")
                
# args = Config()

# Feature extraction

In [18]:
def process_image(image, args=args):
    if args.DES_TYPE == "HOG":
        fd = hog(image, block_norm='L2', pixels_per_cell=args.PIXELS_PER_CELL)
    elif args.DES_TYPE == "LBP":
        fd = lbp(image, args.LBP_POINTS, args.LBP_RADIUS)
    elif args.DES_TYPE == "HAAR":
        fd = haar_like_feature(integral_image(image), 0, 0, 5, 5, 'type-3-x')
    else:
        raise KeyError("==> The Processing method does not exist!")
    return fd

def extract_features(args=args):
    if os.path.exists(args.DIR_PATHS["POS_FEAT_PH"]):
        shutil.rmtree(args.DIR_PATHS["POS_FEAT_PH"])
    if os.path.exists(args.DIR_PATHS["NEG_FEAT_PH"]):
        shutil.rmtree(args.DIR_PATHS["NEG_FEAT_PH"])
    os.makedirs(args.DIR_PATHS["POS_FEAT_PH"])
    os.makedirs(args.DIR_PATHS["NEG_FEAT_PH"])
    
    print("==> Calculating the descriptors for the positive samples and saving them")
    for im_path in tqdm(glob.glob(os.path.join(args.DIR_PATHS["POS_IMG_PH"], "*"))):
        im = imread(im_path, as_grey=True)
        fd = process_image(im)
        fd_name = os.path.split(im_path)[1].split(".")[0] + ".feat"
        fd_path = os.path.join(args.DIR_PATHS["POS_FEAT_PH"], fd_name)
        joblib.dump(fd, fd_path)
    print("==> Positive features saved in {}".format(args.DIR_PATHS["POS_FEAT_PH"]))

    print("==> Calculating the descriptors for the negative samples and saving them")
    for im_path in tqdm(glob.glob(os.path.join(args.DIR_PATHS["NEG_IMG_PH"], "*"))):
        im = imread(im_path, as_grey=True)
        fd = process_image(im)
        fd_name = os.path.split(im_path)[1].split(".")[0] + ".feat"
        fd_path = os.path.join(args.DIR_PATHS["NEG_FEAT_PH"], fd_name)
        joblib.dump(fd, fd_path)
    print("==> Negative features saved in {}".format(args.DIR_PATHS["NEG_FEAT_PH"]))
    print("==> Completed calculating features from training images")

# Train the Classifier

In [19]:
def train_classifier(args=args):
    fds = []
    labels = []
    print("==> Loading the positive features")
    for feat_path in tqdm(glob.glob(os.path.join(args.DIR_PATHS["POS_FEAT_PH"], "*.feat"))):
        fd = joblib.load(feat_path)
        fds.append(fd.reshape(-1))
        labels.append(1)

    print("==> Load the negative features")
    for feat_path in tqdm(glob.glob(os.path.join(args.DIR_PATHS["NEG_FEAT_PH"], "*.feat"))):
        fd = joblib.load(feat_path)
        fds.append(fd.reshape(-1))
        labels.append(0)

    if args.CLF_TYPE is "LIN_SVM":
        clf = LinearSVC()
        print("==> Training a Linear SVM Classifier")
        clf.fit(fds, labels)
        joblib.dump(clf, args.MODEL_PH)
        print("==> Classifier saved to {}".format(args.MODEL_PH))
    elif args.CLF_TYPE is "MLP":
        clf = MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(16, 32, 64), random_state=1)
        print("==> Training a Multi Layer Classifier")
        clf.fit(fds, labels)
        joblib.dump(clf, args.MODEL_PH)
        print("==> Classifier saved to {}".format(args.MODEL_PH))

# Perform Non-Maxima Suppression

In [20]:
def overlapping_area(detection_1, detection_2):
    """
        Function to calculate overlapping area'si
        `detection_1` and `detection_2` are 2 detections whose area
        of overlap needs to be found out.
        Each detection is list in the format ->
        [x-top-left, y-top-left, confidence-of-detections, width-of-detection, height-of-detection]
        The function returns a value between 0 and 1,
        which represents the area of overlap.
        0 is no overlap and 1 is complete overlap.
        Area calculated from ->
        http://math.stackexchange.com/questions/99565/simplest-way-to-calculate-the-intersect-area-of-two-rectangles
    """
    # Calculate the x-y co-ordinates of the rectangles
    x1_tl = detection_1[0]
    x2_tl = detection_2[0]
    x1_br = detection_1[0] + detection_1[3]
    x2_br = detection_2[0] + detection_2[3]
    y1_tl = detection_1[1]
    y2_tl = detection_2[1]
    y1_br = detection_1[1] + detection_1[4]
    y2_br = detection_2[1] + detection_2[4]
    # Calculate the overlapping Area
    x_overlap = max(0, min(x1_br, x2_br) - max(x1_tl, x2_tl))
    y_overlap = max(0, min(y1_br, y2_br) - max(y1_tl, y2_tl))
    overlap_area = x_overlap * y_overlap
    area_1 = detection_1[3] * detection_2[4]
    area_2 = detection_2[3] * detection_2[4]
    total_area = area_1 + area_2 - overlap_area
    return overlap_area / float(total_area)


def nms(detections, threshold=.5):
    """
        This function performs Non-Maxima Suppression.
        `detections` consists of a list of detections.
        Each detection is in the format ->
        [x-top-left, y-top-left, confidence-of-detections, width-of-detection, height-of-detection]
        If the area of overlap is greater than the `threshold`,
        the area with the lower confidence score is removed.
        The output is a list of detections.
    """
    if len(detections) == 0:
        return []
    # Sort the detections based on confidence score
    detections = sorted(detections, key=lambda detections: detections[2],
                        reverse=True)
    new_detections = [] # Unique detections will be appended to this list
    new_detections.append(detections[0]) # Append the first detection
    del detections[0] # Remove the detection from the original list
    """
        For each detection, calculate the overlapping area
        and if area of overlap is less than the threshold set
        for the detections in `new_detections`, append the 
        detection to `new_detections`.
        In either case, remove the detection from `detections` list.
    """
    for index, detection in enumerate(detections):
        for new_detection in new_detections:
            if overlapping_area(detection, new_detection) > threshold:
                del detections[index]
                break
        else:
            new_detections.append(detection)
            del detections[index]
    return new_detections

# Test the classifier

In [21]:
def resize_by_short(img, short_len=256):
    print(img.size)
    (x, y) = img.size
    if x > y:
        y_s = short_len
        x_s = int(x * y_s / y)
        img = img.resize((x_s, y_s))
    else:
        x_s = short_len
        y_s = int(y * x_s / x)
        img = img.resize((x_s, y_s))
    return img


def sliding_window(image, window_size, step_size):
    """
        This function returns a patch of the input image `image` of size equal
        to `window_size`. The first image returned top-left co-ordinates (0, 0)
        and are increment in both x and y directions by the `step_size` supplied.
        So, the input parameters are -
        * `image` - Input Image
        * `window_size` - Size of Sliding Window
        * `step_size` - Incremented Size of Window

        The function returns a tuple -
        (x, y, im_window)
        where
        * x is the top-left x co-ordinate
        * y is the top-left y co-ordinate
        * im_window is the sliding window image
    """
    for y in range(0, image.shape[0], step_size[1]):
        for x in range(0, image.shape[1], step_size[0]):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])


def test_classifier(args=args):
    for im_path in [os.path.join(args.DIR_PATHS["TEST_IMG_DIR_PH"], i) for i in os.listdir(args.DIR_PATHS["TEST_IMG_DIR_PH"]) if not i.startswith('.')]:
        # Read the Image
        im = Image.open(im_path).convert('L')
        im = np.array(resize_by_short(im))

        clf = joblib.load(args.MODEL_PH) # Load the classifier
        detections = [] # List to store the detections
        scale = 0 # The current scale of the image

        # Downscale the image and iterate
        for im_scaled in pyramid_gaussian(im, downscale=args.DOWNSCALE):
            cd = [] # This list contains detections at the current scale
            # If the width or height of the scaled image is less than
            # the width or height of the window, then end the iterations.
            if im_scaled.shape[0] < args.MIN_WDW_SIZE[1] or im_scaled.shape[1] < args.MIN_WDW_SIZE[0]:
                break
            for (x, y, im_window) in sliding_window(im_scaled, args.MIN_WDW_SIZE, args.STEP_SIZE):
                if im_window.shape[0] != args.MIN_WDW_SIZE[1] or im_window.shape[1] != args.MIN_WDW_SIZE[0]:
                    continue
                # Calculate the HOG features
                fd = process_image(im_window).reshape([1, -1])
                pred = clf.predict(fd)
                if pred == 1:
                    if args.IF_PRINT: print("==> Detection:: Location -> ({}, {})".format(x, y))
                    if args.CLF_TYPE is "LIN_SVM":
                        if args.IF_PRINT: print("==> Scale ->  {} Confidence Score {} \n".format(scale, clf.decision_function(fd)))
                        detections.append((x, y, clf.decision_function(fd),
                                           int(args.MIN_WDW_SIZE[0] * (args.DOWNSCALE ** scale)),
                                           int(args.MIN_WDW_SIZE[1] * (args.DOWNSCALE ** scale))))
                    elif args.CLF_TYPE is "MLP":
                        if args.IF_PRINT: print("==> Scale ->  {} Confidence Score {} \n".format(scale, clf.predict_proba(fd)[0][1]))#clf.decision_function(fd)))
                        detections.append((x, y, clf.predict_proba(fd)[0][1],
                                           int(args.MIN_WDW_SIZE[0] * (args.DOWNSCALE ** scale)),
                                           int(args.MIN_WDW_SIZE[1] * (args.DOWNSCALE ** scale))))
                    cd.append(detections[-1])

                # If visualize is set to true, display the working of the sliding window
                if args.VISUALIZE:
                    clone = im_scaled.copy()
                    for x1, y1, _, _, _ in cd:
                        # Draw the detections at this scale
                        cv2.rectangle(clone, (x1, y1), (x1 + im_window.shape[1], y1 +
                                                        im_window.shape[0]), (0, 0, 0), thickness=2)
                    cv2.rectangle(clone, (x, y), (x + im_window.shape[1], y +
                                                  im_window.shape[0]), (255, 255, 255), thickness=2)
                    cv2.imshow("Sliding Window in Progress", clone)
                    cv2.waitKey(30)

            # Move the the next scale
            scale += 1

        # Display the results before performing NMS
        clone = im.copy()

        # Draw the detections
        for (x_tl, y_tl, _, w, h) in detections:
            cv2.rectangle(im, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 0), thickness=2)

        detections_fin = nms(detections, args.THRESHOLD) # Perform Non Maxima Suppression

        # Display the results after performing NMS
        for (x_tl, y_tl, _, w, h) in detections_fin:
            # Draw the detections
            cv2.rectangle(clone, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 0), thickness=2)
            
        if args.VISUALIZE:
            cv2.imshow("Final Detections after applying NMS", clone)
            
#         print(os.path.split(im_path))
        print(os.path.join(args.DIR_PATHS['PRED_SAVE_PH'], os.path.split(im_path)[1]))
        cv2.imwrite(os.path.join(args.DIR_PATHS['PRED_SAVE_PH'], os.path.split(im_path)[1]), clone)

In [22]:
# extract_features()
# train_classifier()
# test_classifier()

# Grid Search

In [24]:
#         self.MIN_WDW_SIZE = [64, 64]
#         self.STEP_SIZE = [12, 12]
#         self.ORIENTATIONS = 9
#         self.PIXELS_PER_CELL = [3, 3]
#         self.CELLS_PER_BLOCK = [3, 3]
import warnings
warnings.filterwarnings("ignore")

# def fxn():
#     warnings.warn("deprecated", DeprecationWarning)

for step_size in range(8,36,4):
    for pixels_per_cell in range(3,10,1):
        for cells_per_block in range(3, 10, 1):
            args = Config()
            args.STEP_SIZE = [step_size, step_size]
            args.CELLS_PER_BLOCK = [cells_per_block, cells_per_block]
            args.PIXELS_PER_CELL = [pixels_per_cell, pixels_per_cell]
            args.PROJECT_ID = args.PROJECT_ID + "_SS_" + str(step_size) +\
                    "_CPB_" + str(cells_per_block) + "_PPC_" + str(pixels_per_cell)
            args.update_names()
            print(args.PROJECT_ID, args.DIR_PATHS)
            args.mk_new_dirs()
            extract_features(args=args)
            train_classifier(args=args)
            test_classifier(args=args)
            if not args.KEEP_FEAT:
                shutil.rmtree(args.DIR_PATHS['NEG_FEAT_PH'])
                shutil.rmtree(args.DIR_PATHS['POS_FEAT_PH'])


  0%|          | 12/9731 [00:00<01:23, 116.30it/s]

New_Vedio_New_NegHOG_LIN_SVM_SS_8_CPB_3_PPC_3 {'POS_FEAT_PH': './source/features/New_Vedio_New_NegHOG_LIN_SVM_SS_8_CPB_3_PPC_3/pos', 'NEG_FEAT_PH': './source/features/New_Vedio_New_NegHOG_LIN_SVM_SS_8_CPB_3_PPC_3/neg', 'MODEL_DIR_PH': './source/models/New_Vedio_New_NegHOG_LIN_SVM_SS_8_CPB_3_PPC_3', 'PRED_SAVE_PH': './source/predictions/New_Vedio_New_NegHOG_LIN_SVM_SS_8_CPB_3_PPC_3', 'POS_IMG_PH': './source/images/pos', 'NEG_IMG_PH': './source/images/neg', 'TEST_IMG_DIR_PH': './source/test_images'}
==> Calculating the descriptors for the positive samples and saving them



  4%|▎         | 349/9731 [00:03<01:25, 109.50it/s]
Exception in thread Thread-14:
Traceback (most recent call last):
  File "/Users/miracle/anaconda2/envs/py3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/miracle/anaconda2/envs/py3/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/miracle/anaconda2/envs/py3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  1%|          | 11/2000 [00:00<00:18, 106.34it/s]

==> Positive features saved in ./source/features/New_Vedio_New_NegHOG_LIN_SVM_SS_8_CPB_3_PPC_3/pos
==> Calculating the descriptors for the negative samples and saving them


  0%|          | 0/2000 [00:00<?, ?it/s]

...

 11%|█▏        | 228/2000 [00:00<00:00, 2275.47it/s]

==> Load the negative features


100%|██████████| 2000/2000 [00:01<00:00, 1328.56it/s]


==> Training a Linear SVM Classifier
==> Classifier saved to ./source/models/New_Vedio_New_NegHOG_LIN_SVM_SS_8_CPB_8_PPC_4/svm.model
(2320, 2320)
./source/predictions/New_Vedio_New_NegHOG_LIN_SVM_SS_8_CPB_8_PPC_4/test_9.JPG
(1920, 1920)
./source/predictions/New_Vedio_New_NegHOG_LIN_SVM_SS_8_CPB_8_PPC_4/test_8.JPG
(2576, 1932)


KeyboardInterrupt: 

# Pre-processing of Video-Generated images

In [26]:
prefix = './output3/'
outdir = ['./new_video/pos', './new_video/neg']
dirs = os.listdir(prefix)
negs = []
poss = []
for adir in dirs:
    if adir.startswith('neg'):
        negs.append(os.path.join(prefix, adir))
    elif adir.startswith('pos'):
        poss.append(os.path.join(prefix, adir))

In [28]:
for adir in poss:
    for img in os.listdir(adir):
#         print(os.path.join(adir, img), os.path.join(outdir[0], img))
        os.rename(os.path.join(adir, img), os.path.join(outdir[0], os.path.split(adir)[1]+'_'+img))

for adir in negs:
    for img in os.listdir(adir):
#         print(os.path.join(adir, img), os.path.join(outdir[1], img))
        os.rename(os.path.join(adir, img), os.path.join(outdir[1], os.path.split(adir)[1]+'_'+img))